## Top tag : 0.9 , 120 250 
## B tag : particle net tight 0.6734


## BKG

In [1]:
#!/usr/bin/env python3
import os
import re
import glob
import csv

# 1) 로그 파일이 들어있는 폴더
LOG_DIR = "/data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/bkg/condorfile/out"

# 2) 결과를 담을 리스트
rows = []

# 3) 각 로그 파일 순회
for logpath in glob.glob(os.path.join(LOG_DIR, "*.log")):
    name = os.path.splitext(os.path.basename(logpath))[0]  # 파일명 베이스
    with open(logpath, 'r') as f:
        text = f.read()

    # 4) 정규표현식으로 숫자 추출
    m_pass = re.search(r"Passed events\s*:\s*([0-9]+)", text)
    m_weight = re.search(r"Per-event weight\s*:\s*([0-9.eE\+\-]+)", text)
    m_exp = re.search(r"Expected yield\s*:\s*([0-9.eE\+\-]+)", text)

    # 5) 모두 찾았을 때만 추가
    if m_pass and m_weight and m_exp:
        passed      = int(m_pass.group(1))
        weight      = float(m_weight.group(1))
        expected    = float(m_exp.group(1))
        rows.append({
            "sample": name,
            "Passed events": passed,
            "Per-event weight": weight,
            "Expected yield": expected
        })
    else:
        print(f"Skipping {name}: incomplete data")

# 6) Expected yield 기준으로 내림차순 정렬
rows.sort(key=lambda x: x["Expected yield"], reverse=True)

# → 여기서 각 컬럼 합계 계산
total_passed   = sum(r["Passed events"]      for r in rows)
total_weight   = sum(r["Per-event weight"]   for r in rows)
total_expected = sum(r["Expected yield"]      for r in rows)

# → 합계 행을 추가
rows.append({
    "sample":          "Total",
    "Passed events":   total_passed,
    "Per-event weight": total_weight,
    "Expected yield":  total_expected
})

# 7) CSV로 저장 (Total 행 포함)
out_csv = "summary.csv"
with open(out_csv, 'w', newline='') as csvfile:
    fieldnames = ["sample", "Passed events", "Per-event weight", "Expected yield"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)

print(f"Wrote {len(rows)} entries (including total row) to {out_csv}")

Skipping SingleMuon_C: incomplete data
Skipping SingleMuon_B: incomplete data
Skipping JetMET_C: incomplete data
Skipping EGamma_D: incomplete data
Skipping JetMET_D: incomplete data
Skipping EGamma_B: incomplete data
Skipping Tau_B: incomplete data
Skipping Jpsito2Mu: incomplete data
Skipping Tau_C: incomplete data
Skipping Muon_C: incomplete data
Skipping Tau_D: incomplete data
Skipping EGamma_C: incomplete data
Skipping Muon_D: incomplete data
Skipping JetMET_B: incomplete data
Wrote 47 entries (including total row) to summary.csv


## SIG

In [2]:
#!/usr/bin/env python3
import os
import re
import glob
import csv
import math

# 0) background summary.csv 에서 B 값 읽어오기
BG_SUMMARY = "summary.csv"  # 필요에 따라 경로를 조정하세요
with open(BG_SUMMARY, newline='') as f_bg:
    reader_bg = csv.DictReader(f_bg)
    bg_rows = list(reader_bg)
    # 마지막 행의 "Expected yield" 를 B 로 설정
    B = float(bg_rows[-1]["Expected yield"])

# 1) 로그 파일이 들어있는 폴더
LOG_DIR = "/data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/sig/condorfile/out"

rows = []
for logpath in glob.glob(os.path.join(LOG_DIR, "*.log")):
    name = os.path.splitext(os.path.basename(logpath))[0]
    with open(logpath, 'r') as f:
        text = f.read()

    m_pass   = re.search(r"Passed events\s*:\s*([0-9]+)", text)
    m_weight = re.search(r"Per-event weight\s*:\s*([0-9.eE\+\-]+)", text)
    m_exp    = re.search(r"Expected yield\s*:\s*([0-9.eE\+\-]+)", text)

    if m_pass and m_weight and m_exp:
        passed   = int(m_pass.group(1))
        weight   = float(m_weight.group(1))
        expected = float(m_exp.group(1))

        # Z = S / sqrt(B)
        Z = expected / math.sqrt(B) if B > 0 else float('nan')
        # p-value = 1 - Phi(Z) = 0.5*erfc(Z/sqrt(2))
        p_value = 0.5 * math.erfc(Z / math.sqrt(2))

        rows.append({
            "sample":           name,
            "Passed events":    passed,
            "Per-event weight": weight,
            "Expected yield":   expected,
            "S_over_sqrtB":     Z,
            "p_value":          p_value
        })
    else:
        print(f"Skipping {name}: incomplete data")

# 정렬 (선택)
rows.sort(key=lambda x: x["Expected yield"], reverse=True)

# 7) CSV로 저장 (새 컬럼 포함)
out_csv = "sig_summary.csv"
with open(out_csv, 'w', newline='') as csvfile:
    fieldnames = [
        "sample",
        "Passed events",
        "Per-event weight",
        "Expected yield",
        "S_over_sqrtB",
        "p_value"
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)

print(f"Wrote {len(rows)} entries to {out_csv}")


Wrote 8 entries to sig_summary.csv


## Bin by bin collector

#### bkg

In [6]:
import os
import glob
import pandas as pd

# ——————————————————————————————
# 1) 베이스 디렉토리 설정
base_dir = "/data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/bkg/condorfile"

# 2) 처리할 카운트 파일 종류 리스트
file_types = ["wrmass", "mll", "ptmu1", "ptmu2", "ptt", "ptb"]

for ft in file_types:
    dfs = []
    # 3) 각 result_* 폴더 안의 {ft}_counts.csv 파일들 읽어오기
    pattern = os.path.join(base_dir, "result_*", f"{ft}_counts.csv")
    for csv_path in glob.glob(pattern):
        df = pd.read_csv(csv_path)
        # 4) sample 컬럼에 폴더 이름 추가
        sample = os.path.basename(os.path.dirname(csv_path))
        df.insert(0, "sample", sample)
        dfs.append(df)
    
    if not dfs:
        print(f"[{ft}] No files found under pattern: {pattern}")
        continue

    # 5) 모든 df 합치기
    combined = pd.concat(dfs, ignore_index=True)

    # 6) 자동으로 첫 3개 컬럼 인식: index, pivot_col, value
    index_col = combined.columns[0]   # sample
    pivot_col = combined.columns[1]   # pt_bin or mass_bin 등
    value_col = combined.columns[2]   # count

    # 7) pivot & 결측치는 0으로
    pivot = combined.pivot(
        index=index_col,
        columns=pivot_col,
        values=value_col
    ).fillna(0)

    # 8) 결과 파일명 & 저장
    outname = os.path.join(
        base_dir,
        f"/data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/bkg_{ft}_counts_pivot.csv"
    )
    pivot.to_csv(outname, index=True)
    print(f"[{ft}] Saved pivot CSV: {outname} ({pivot.shape[0]} samples × {pivot.shape[1]} bins)")

[wrmass] Saved pivot CSV: /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/bkg_wrmass_counts_pivot.csv (46 samples × 10 bins)
[mll] Saved pivot CSV: /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/bkg_mll_counts_pivot.csv (46 samples × 13 bins)
[ptmu1] Saved pivot CSV: /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/bkg_ptmu1_counts_pivot.csv (46 samples × 12 bins)
[ptmu2] Saved pivot CSV: /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/bkg_ptmu2_counts_pivot.csv (46 samples × 12 bins)
[ptt] Saved pivot CSV: /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4

#### sig

In [7]:
import os
import glob
import pandas as pd

# ——————————————————————————————
# 1) 베이스 디렉토리 설정
base_dir = "/data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/sig/condorfile"

# 2) 처리할 카운트 파일 종류 리스트
file_types = ["wrmass", "mll", "ptmu1", "ptmu2", "ptt", "ptb"]

for ft in file_types:
    dfs = []
    # 3) 각 result_* 폴더 안의 {ft}_counts.csv 파일들 읽어오기
    pattern = os.path.join(base_dir, "result_*", f"{ft}_counts.csv")
    for csv_path in glob.glob(pattern):
        df = pd.read_csv(csv_path)
        # 4) sample 컬럼에 폴더 이름 추가
        sample = os.path.basename(os.path.dirname(csv_path))
        df.insert(0, "sample", sample)
        dfs.append(df)
    
    if not dfs:
        print(f"[{ft}] No files found under pattern: {pattern}")
        continue

    # 5) 모든 df 합치기
    combined = pd.concat(dfs, ignore_index=True)

    # 6) 자동으로 첫 3개 컬럼 인식: index, pivot_col, value
    index_col = combined.columns[0]   # sample
    pivot_col = combined.columns[1]   # pt_bin or mass_bin 등
    value_col = combined.columns[2]   # count

    # 7) pivot & 결측치는 0으로
    pivot = combined.pivot(
        index=index_col,
        columns=pivot_col,
        values=value_col
    ).fillna(0)

    # 8) 결과 파일명 & 저장
    outname = os.path.join(
        base_dir,
        f"/data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/sig_{ft}_counts_pivot.csv"
    )
    pivot.to_csv(outname, index=True)
    print(f"[{ft}] Saved pivot CSV: {outname} ({pivot.shape[0]} samples × {pivot.shape[1]} bins)")

[wrmass] Saved pivot CSV: /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/sig_wrmass_counts_pivot.csv (8 samples × 10 bins)
[mll] Saved pivot CSV: /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/sig_mll_counts_pivot.csv (8 samples × 13 bins)
[ptmu1] Saved pivot CSV: /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/sig_ptmu1_counts_pivot.csv (8 samples × 12 bins)
[ptmu2] Saved pivot CSV: /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/sig_ptmu2_counts_pivot.csv (8 samples × 12 bins)
[ptt] Saved pivot CSV: /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK

In [8]:
base_dir = "/data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0"

### plot

In [9]:
import os, glob, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

out_dir  = base_dir

def parse_bin_label(lbl):
    parts = re.split(r"[–\-]", lbl)
    low  = float(parts[0])
    high = np.inf if parts[1].lower() in ("inf","infinity") else float(parts[1])
    return low, high, lbl

def extract_MWR(sample):
    m = re.search(r"MWR-(\d+)", sample)
    return int(m.group(1)) if m else None

for sig_path in glob.glob(os.path.join(base_dir, "sig_*_counts_pivot.csv")):
    var = os.path.basename(sig_path).split("_")[1]
    df_sig = pd.read_csv(sig_path, index_col="sample")
    bkg_path = os.path.join(base_dir, f"bkg_{var}_counts_pivot.csv")
    if not os.path.exists(bkg_path): continue
    df_bkg = pd.read_csv(bkg_path, index_col="sample")

    # bin parsing & sort
    bin_infos = sorted((parse_bin_label(lbl) for lbl in df_sig.columns),
                       key=lambda x: x[0])
    lows    = np.array([b[0] for b in bin_infos])
    highs   = np.array([b[1] for b in bin_infos])
    labels  = [b[2] for b in bin_infos]
    widths  = highs - lows
    widths[np.isinf(widths)] = widths[~np.isinf(widths)][-1]
    centers = (lows + highs) / 2

    # background stacks
    totals = df_bkg.sum(axis=1).sort_values(ascending=False)
    top2, others = totals.index[:2].tolist(), totals.index[2:].tolist()
    bkg_other = df_bkg.loc[others].sum().reindex(labels).values
    bkg_top2  = df_bkg.loc[top2[1]].reindex(labels).values
    bkg_top1  = df_bkg.loc[top2[0]].reindex(labels).values

    finite_highs = highs[np.isfinite(highs)]
    max_high     = finite_highs.max() if len(finite_highs) else highs[-1]

    for sample, sig_row in df_sig.iterrows():
        sig_vals   = sig_row.reindex(labels).values
        base_stack = bkg_other + bkg_top2 + bkg_top1
        y_sig      = base_stack + sig_vals

        if np.max(y_sig) < 1.0:
            print(f"✖ Skipped {sample} ({var}): all < 1")
            continue

        # x-axis end
        if var == "wrmass":
            MWR = extract_MWR(sample) or max_high
            extended_end = MWR + 3000
            ext_low   = lows[-1]
            ext_width = extended_end - ext_low
            last_val  = y_sig[-1]
        else:
            last_low  = lows[-1]
            last_high = highs[-1]
            extended_end = last_low + 1000 if np.isinf(last_high) else last_high

        fig, ax = plt.subplots(figsize=(8,5))
        bottom = np.zeros_like(lows)

        # plot background
        ax.bar(lows, bkg_other, widths, bottom=0, label="other bkg", align="edge")
        bottom += bkg_other
        ax.bar(lows, bkg_top2, widths, bottom=bottom, label=top2[1], align="edge")
        bottom += bkg_top2
        ax.bar(lows, bkg_top1, widths, bottom=bottom, label=top2[0], align="edge")

        # extension for wrmass
        if var == "wrmass":
            ax.bar(ext_low, bkg_other[-1], ext_width, bottom=0,
                   color=ax.containers[0].patches[0].get_facecolor(), align="edge")
            ax.bar(ext_low, bkg_top2[-1], ext_width, bottom=bkg_other[-1],
                   color=ax.containers[1].patches[0].get_facecolor(), align="edge")
            ax.bar(ext_low, bkg_top1[-1], ext_width, bottom=bkg_other[-1]+bkg_top2[-1],
                   color=ax.containers[2].patches[0].get_facecolor(), align="edge")
            # extend y_sig
            lows_ext = np.append(lows, ext_low)
            highs_ext= np.append(highs, extended_end)
            y_sig_ext= np.append(y_sig, last_val)
        else:
            lows_ext  = lows.copy()
            highs_ext = highs.copy()
            y_sig_ext = y_sig.copy()

        # overlay signal as dashed-rectangle
        for low, high, top in zip(lows_ext, highs_ext, y_sig_ext):
            rect = Rectangle(
                (low, 0),               # x,y of lower-left
                high - low,             # width
                top,                    # height
                fill=False,
                linestyle='--',
                linewidth=1.5,
                edgecolor='black',
                zorder=5
            )
            ax.add_patch(rect)

        # final formatting
        ax.set_xlim(0, extended_end)
        ax.set_xticks(np.arange(0, extended_end+1, 500))
        ax.set_xlabel(var)
        ax.set_ylabel("Weighted count")
        ax.set_title(f"Stacked {var} for {sample}")
        ax.legend(loc="upper right", fontsize="small")
        plt.tight_layout()
        # 6) y축 로그 스케일
        ax.set_yscale('log')
        outpath = os.path.join(out_dir, f"{sample}_{var}_stacked.png")
        fig.savefig(outpath)
        plt.close(fig)
        print(f"✔ Saved {outpath}")

✔ Saved /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/result_WRtoNMutoMuMuTB-HadTop_MWR-1000_MN-700_13p6TeV_wrmass_stacked.png
✔ Saved /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/result_WRtoNMutoMuMuTB-HadTop_MWR-3000_MN-1300_13p6TeV_wrmass_stacked.png
✔ Saved /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/result_WRtoNMutoMuMuTB-HadTop_MWR-3000_MN-2100_13p6TeV_wrmass_stacked.png
✖ Skipped result_WRtoNMutoMuMuTB-HadTop_MWR-3000_MN-2900_13p6TeV (wrmass): all < 1
✔ Saved /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/result_WRtoNMutoMuMuTB-HadTop_MWR-5000_MN-2500_13p6TeV_wrmass_st

In [14]:
import os, glob, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

base_dir = "/data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0"
out_dir  = base_dir

def parse_bin_label(lbl):
    parts = re.split(r"[–\-]", lbl)
    low  = float(parts[0])
    high = np.inf if parts[1].lower() in ("inf","infinity") else float(parts[1])
    return low, high, lbl

def extract_MWR(sample):
    m = re.search(r"MWR-(\d+)", sample)
    return int(m.group(1)) if m else None

for sig_path in glob.glob(os.path.join(base_dir, "sig_*_counts_pivot.csv")):
    var = os.path.basename(sig_path).split("_")[1]
    df_sig = pd.read_csv(sig_path, index_col="sample")
    bkg_file = os.path.join(base_dir, f"bkg_{var}_counts_pivot.csv")
    if not os.path.exists(bkg_file): continue
    df_bkg = pd.read_csv(bkg_file, index_col="sample")

    # --- bin 세팅 ---
    bin_infos = sorted((parse_bin_label(lbl) for lbl in df_sig.columns),
                       key=lambda x: x[0])
    lows   = np.array([b[0] for b in bin_infos])
    highs  = np.array([b[1] for b in bin_infos])
    widths = highs - lows
    widths[np.isinf(widths)] = widths[~np.isinf(widths)][-1]
    labels = [b[2] for b in bin_infos]

    # background 스택 값
    totals = df_bkg.sum(axis=1).sort_values(ascending=False)
    top2, others = totals.index[:2].tolist(), totals.index[2:].tolist()
    bkg_other = df_bkg.loc[others].sum().reindex(labels).values
    bkg_top2  = df_bkg.loc[top2[1]].reindex(labels).values
    bkg_top1  = df_bkg.loc[top2[0]].reindex(labels).values

    # x축 끝점 계산 (모든 시그널에서 최대값 찾기)
    finite_highs = highs[np.isfinite(highs)]
    max_high = finite_highs.max() if len(finite_highs) else highs[-1]
    
    if var == "wrmass":
        # 모든 시그널의 MWR 값을 구해서 최대값 찾기
        all_MWRs = [extract_MWR(s) for s in df_sig.index if extract_MWR(s) is not None]
        if all_MWRs:
            max_MWR = max(all_MWRs)
            x_end = max_MWR + 3000
        else:
            x_end = max_high + 3000
    else:
        last_high = highs[-1]
        x_end = lows[-1] + 1000 if np.isinf(last_high) else last_high

    xticks = np.arange(0, x_end+1, 500)

    # --- background 스택 플롯 ---
    fig, ax = plt.subplots(figsize=(10,6))
    bottom = np.zeros_like(lows)
    ax.bar(lows, bkg_other, widths, bottom=bottom, label="other bkg", align="edge")
    bottom += bkg_other
    ax.bar(lows, bkg_top2, widths,  bottom=bottom, label=top2[1], align="edge")
    bottom += bkg_top2
    ax.bar(lows, bkg_top1, widths,  bottom=bottom, label=top2[0], align="edge")

    # --- 모든 시그널 오버레이 (가로선만) ---
    cmap = plt.get_cmap("tab10")
    for i, (sample, sig_row) in enumerate(df_sig.iterrows()):
        # signal 높이
        sig_vals   = sig_row.reindex(labels).values
        heights    = bottom + sig_vals

        color = cmap(i % cmap.N)
        
        # 각 bin에 대해 가로선만 그리기
        for j, (low, high, height) in enumerate(zip(lows, highs, heights)):
            if np.isinf(high):
                # 마지막 bin이 무한대인 경우, 모든 시그널에 대해 동일한 x_end 사용
                high = x_end
            
            # 가로선만 그리기 (세로선 없음)
            if j == 0:  # 첫 번째 선에만 label 추가
                ax.plot([low, high], [height, height], 
                       color=color, linewidth=2, label=sample)
            else:
                ax.plot([low, high], [height, height], 
                       color=color, linewidth=2)

    # --- 축/레이블 ---
    ax.set_xlim(0, x_end)
    ax.set_xticks(xticks)
    ax.set_xticklabels([f"{int(x)}" for x in xticks], rotation=45)
    ax.set_xlabel(var)
    ax.set_ylabel("Weighted count")
    ax.set_title(f"Stacked backgrounds + all signals ({var})")
    ax.legend(loc="upper right", fontsize="small", ncol=2)
    plt.tight_layout()
    
    # y축 로그 스케일 및 y축 상한 조정
    ax.set_yscale('log')
    y_min, y_max = ax.get_ylim()
    ax.set_ylim(y_min, y_max * 100)  # log 스케일에서 약 2단계(10^2) 상향
    
    # --- 저장 ---
    outpath = os.path.join(out_dir, f"{var}_all_signals_stacked.png")
    fig.savefig(outpath)
    plt.close(fig)
    print(f"✔ Saved {outpath}")

✔ Saved /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/wrmass_all_signals_stacked.png
✔ Saved /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/mll_all_signals_stacked.png
✔ Saved /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/ptmu1_all_signals_stacked.png
✔ Saved /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/ptmu2_all_signals_stacked.png
✔ Saved /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/using_matrix/condor/1AK4_1AK8/result/btag_PN_tight_hlt_30/WR_cut_0/top_0.9_120_250/mll0pt0eta0/ptt_all_signals_stacked.png
✔ Saved /data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/